<a href="https://colab.research.google.com/github/aashrithresearch/endometrial-cancer/blob/main/Untitled47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q datasets transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

dataset = load_dataset('/content/drive/MyDrive/ep classification')

Resolving data files:   0%|          | 0/1968 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [5]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1771
})

In [6]:
train_ds.features

{'image': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['Endometrial Polyp', 'Normal Endometrium'], id=None)}

In [7]:
train_ds[0]['label']

1

In [8]:
id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

{0: 'Endometrial Polyp', 1: 'Normal Endometrium'}

In [9]:
id2label[train_ds[0]['label']]

'Normal Endometrium'

In [10]:
from transformers import ViTImageProcessor

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [11]:
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

In [12]:
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)

In [13]:
train_ds[:2]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>],
 'label': [1, 0],
 'pixel_values': [tensor([[[ 0.4510,  0.4667,  0.4588,  ...,  0.3804,  0.3490,  0.2157],
           [ 0.4667,  0.4824,  0.4667,  ...,  0.4431,  0.3490,  0.1216],
           [ 0.4667,  0.4745,  0.4667,  ...,  0.4588,  0.3882,  0.0980],
           ...,
           [-0.1294, -0.2078, -0.4353,  ...,  0.4902,  0.4824,  0.4824],
           [-0.0980, -0.1373, -0.3804,  ...,  0.4588,  0.4745,  0.4745],
           [-0.3098, -0.3804, -0.4510,  ...,  0.3647,  0.3882,  0.4118]],
  
          [[ 0.2314,  0.2471,  0.2392,  ..., -0.1137, -0.1529, -0.2157],
           [ 0.2471,  0.2627,  0.2471,  ..., -0.0431, -0.1216, -0.2784],
           [ 0.2471,  0.2549,  0.2471,  ..., -0.0196, -0.0588, -0.2784],
           ...,
           [-0.5294, -0.5373, -0.6706,  ...,  0.2000,  0.1529,  0.1216],
           [-0.4588, -0.4431, -0.5922,  ...,  0.1529,  0

In [14]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(train_ds, collate_fn=collate_fn, batch_size=4)

In [15]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k, v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [16]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  id2label=id2label,
                                                  label2id=label2id)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import TrainingArguments, Trainer

metric_name = "accuracy"

args = TrainingArguments(
    f"test-et",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels), precision_recall_fscore_support=precision_recall_fscore_support(predictions, labels, average='macro'))

In [19]:
import torch

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

<ipython-input-19-0bb229cc289f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision Recall Fscore Support
1,No log,0.471702,0.771574,"(0.7416588345864661, 0.7397435897435898, 0.7406757349714788, None)"
2,No log,0.536579,0.746193,"(0.6134281015037594, 0.8333908839779005, 0.6078821656050956, None)"
3,0.457000,0.336391,0.862944,"(0.8539121240601504, 0.8410892210144927, 0.8467721026704693, None)"
4,0.457000,0.418994,0.771574,"(0.7740836466165413, 0.748297147722435, 0.754466140423764, None)"
5,0.457000,0.345040,0.837563,"(0.8189027255639098, 0.8139602128151746, 0.8163170163170164, None)"
6,0.310300,0.477876,0.817259,"(0.7876527255639098, 0.7925328554360813, 0.7899786780383795, None)"
7,0.310300,0.403370,0.847716,"(0.8021029135338346, 0.8410477453580902, 0.8167328206400397, None)"
8,0.310300,0.400786,0.812183,"(0.8001057330827068, 0.7855466130114017, 0.7915343915343915, None)"
9,0.247600,0.480109,0.842640,"(0.8064497180451128, 0.8268796992481203, 0.8151242696697242, None)"
10,0.247600,0.385996,0.868020,"(0.8536184210526316, 0.8481378672218367, 0.8507575757575758, None)"


Trainer is attempting to log a value of "(0.7416588345864661, 0.7397435897435898, 0.7406757349714788, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.6134281015037594, 0.8333908839779005, 0.6078821656050956, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.8539121240601504, 0.8410892210144927, 0.8467721026704693, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.7740836466165413, 0.748297147722435, 0.754466140423764, None)" of type <class 'tuple'> fo

Epoch,Training Loss,Validation Loss,Accuracy,Precision Recall Fscore Support
1,No log,0.471702,0.771574,"(0.7416588345864661, 0.7397435897435898, 0.7406757349714788, None)"
2,No log,0.536579,0.746193,"(0.6134281015037594, 0.8333908839779005, 0.6078821656050956, None)"
3,0.457000,0.336391,0.862944,"(0.8539121240601504, 0.8410892210144927, 0.8467721026704693, None)"
4,0.457000,0.418994,0.771574,"(0.7740836466165413, 0.748297147722435, 0.754466140423764, None)"
5,0.457000,0.345040,0.837563,"(0.8189027255639098, 0.8139602128151746, 0.8163170163170164, None)"
6,0.310300,0.477876,0.817259,"(0.7876527255639098, 0.7925328554360813, 0.7899786780383795, None)"
7,0.310300,0.403370,0.847716,"(0.8021029135338346, 0.8410477453580902, 0.8167328206400397, None)"
8,0.310300,0.400786,0.812183,"(0.8001057330827068, 0.7855466130114017, 0.7915343915343915, None)"
9,0.247600,0.480109,0.842640,"(0.8064497180451128, 0.8268796992481203, 0.8151242696697242, None)"
10,0.247600,0.385996,0.868020,"(0.8536184210526316, 0.8481378672218367, 0.8507575757575758, None)"


Trainer is attempting to log a value of "(0.8420465225563909, 0.8367453157529493, 0.8392773892773893, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.8417528195488722, 0.8445865908552476, 0.8431389896487658, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4450, training_loss=0.20421812421820137, metrics={'train_runtime': 2685.6737, 'train_samples_per_second': 16.486, 'train_steps_per_second': 1.657, 'total_flos': 3.430957840186061e+18, 'train_loss': 0.20421812421820137, 'epoch': 25.0})

In [21]:
trainer.evaluate()

Trainer is attempting to log a value of "(0.8608435150375939, 0.8702386692381872, 0.8652490706319702, None)" of type <class 'tuple'> for key "eval/precision_recall_fscore_support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.39242708683013916,
 'eval_accuracy': 0.883248730964467,
 'eval_precision_recall_fscore_support': (0.8608435150375939,
  0.8702386692381872,
  0.8652490706319702,
  None),
 'eval_runtime': 6.4074,
 'eval_samples_per_second': 30.746,
 'eval_steps_per_second': 7.803,
 'epoch': 25.0}